# CRYPTO-02: Группы и конечные поля

В этом notebook вы реализуете конечные поля GF(p), найдете генераторы циклических групп и поработаете с библиотекой `galois`.

**Темы:**
- Реализация GF(p) вручную
- Операции с библиотекой galois
- Генераторы циклических групп
- Порядки элементов
- Полиномиальные поля GF(2^8)

## 1. Ручная реализация GF(p)

Создадим класс для элемента конечного поля с поддержкой всех операций.

In [ ]:
class FieldElement:
    """Элемент конечного поля GF(p)."""
    
    def __init__(self, value: int, prime: int):
        if prime < 2:
            raise ValueError(f"prime должно быть >= 2, получено {prime}")
        self.value = value % prime
        self.prime = prime
    
    def __repr__(self):
        return f"FieldElement({self.value}, {self.prime})"
    
    def __eq__(self, other):
        if not isinstance(other, FieldElement):
            return False
        return self.value == other.value and self.prime == other.prime
    
    def _check_field(self, other):
        if self.prime != other.prime:
            raise ValueError(f"Разные поля: GF({self.prime}) и GF({other.prime})")
    
    def __add__(self, other):
        self._check_field(other)
        return FieldElement((self.value + other.value) % self.prime, self.prime)
    
    def __sub__(self, other):
        self._check_field(other)
        return FieldElement((self.value - other.value) % self.prime, self.prime)
    
    def __mul__(self, other):
        self._check_field(other)
        return FieldElement((self.value * other.value) % self.prime, self.prime)
    
    def __truediv__(self, other):
        self._check_field(other)
        return self * other.inverse()
    
    def __pow__(self, exp: int):
        if exp < 0:
            return self.inverse() ** (-exp)
        return FieldElement(pow(self.value, exp, self.prime), self.prime)
    
    def inverse(self):
        if self.value == 0:
            raise ValueError("Ноль не имеет обратного элемента")
        return FieldElement(pow(self.value, -1, self.prime), self.prime)
    
    def order(self) -> int:
        """Мультипликативный порядок элемента."""
        if self.value == 0:
            raise ValueError("Порядок нуля не определен")
        current = FieldElement(1, self.prime)
        for i in range(1, self.prime):
            current = current * self
            if current.value == 1:
                return i
        return self.prime - 1

# Тесты
p = 7
a = FieldElement(3, p)
b = FieldElement(5, p)

print(f"a = {a}")
print(f"b = {b}")
print(f"a + b = {a + b}")     # 1
print(f"a * b = {a * b}")     # 1
print(f"a^(-1) = {a.inverse()}")  # 5
print(f"a / b = {a / b}")     # 3 / 5 = 3 * 5^(-1) = 3 * 3 = 2
print(f"a^3 = {a ** 3}")      # 27 % 7 = 6
print(f"ord(a) = {a.order()}") # порядок 3 в Z*_7

## 2. Операции с библиотекой galois

Библиотека `galois` предоставляет эффективную реализацию конечных полей с NumPy-совместимым интерфейсом.

In [ ]:
import galois
import numpy as np

# Создаем поле GF(23)
GF23 = galois.GF(23)
print(f"Поле: {GF23}")
print(f"Порядок: {GF23.order}")
print(f"Характеристика: {GF23.characteristic}")

# Базовые операции
a = GF23(7)
b = GF23(15)
print(f"\n7 + 15 = {a + b} (mod 23)")
print(f"7 * 15 = {a * b} (mod 23)")
print(f"7^(-1) = {GF23(7) ** -1} (mod 23)")

# Таблица умножения
elements = GF23(list(range(23)))
mult_table = np.outer(elements, elements)
print(f"\nТаблица умножения GF(23) (первые 5x5):")
print(mult_table[:5, :5])

## 3. Генераторы циклических групп

Элемент `g` является **генератором** (примитивным корнем) `Z*_p`, если его порядок равен `p - 1`.

In [ ]:
def find_generators(p: int) -> list[int]:
    """Находит все генераторы Z*_p."""
    generators = []
    for g in range(2, p):
        # Генерируем все степени g
        seen = set()
        current = 1
        for _ in range(p - 1):
            current = (current * g) % p
            seen.add(current)
        
        if len(seen) == p - 1:
            generators.append(g)
    
    return generators

# Генераторы для малых простых
for p in [5, 7, 11, 13, 17, 23]:
    gens = find_generators(p)
    print(f"Z*_{p}: генераторы = {gens} (всего {len(gens)} из {p-1} элементов)")

# Свойство: количество генераторов = phi(p-1)
# phi(6) = 2 -> у Z*_7 два генератора: 3 и 5
print(f"\nСтепени генератора g=3 в Z*_7:")
for i in range(7):
    print(f"  3^{i} = {pow(3, i, 7)} (mod 7)")

## 4. Порядки элементов

Порядок элемента `a` — минимальное `k > 0`, такое что `a^k = 1 (mod p)`. По теореме Лагранжа, порядок делит `p - 1`.

In [ ]:
def element_order(a: int, p: int) -> int:
    """Вычисляет мультипликативный порядок a в Z*_p."""
    if a % p == 0:
        raise ValueError(f"{a} = 0 в Z*_{p}")
    
    current = a % p
    for k in range(1, p):
        if current == 1:
            return k
        current = (current * a) % p
    return p - 1

# Порядки всех элементов Z*_13
p = 13
print(f"Порядки элементов Z*_{p} (|Z*_{p}| = {p-1}):")
print(f"Делители {p-1}: ", end="")
divisors = [d for d in range(1, p) if (p - 1) % d == 0]
print(divisors)

order_groups = {}
for a in range(1, p):
    ord_a = element_order(a, p)
    order_groups.setdefault(ord_a, []).append(a)
    print(f"  ord({a:2d}) = {ord_a}")

print(f"\nГруппировка по порядку:")
for order, elements in sorted(order_groups.items()):
    print(f"  Порядок {order:2d}: {elements} ({len(elements)} элементов)")

## Упражнения

### Упражнение 1: Проверка аксиом группы

Напишите функцию `verify_group(elements, operation, modulus)`, которая проверяет все четыре аксиомы группы (замкнутость, ассоциативность, нейтральный элемент, обратные). Протестируйте на `Z*_11`.

In [ ]:
# Ваш код здесь


### Упражнение 2: Дискретный логарифм (brute force)

Реализуйте функцию `discrete_log(g, h, p)`, которая находит `x` такое что `g^x = h (mod p)` перебором. Протестируйте для `g=5, h=8, p=23`. Замерьте время для `p=65537` (простое Ферма).

In [ ]:
# Ваш код здесь


### Упражнение 3: Baby-step Giant-step

Реализуйте алгоритм Baby-step Giant-step для дискретного логарифма.
Сложность: O(sqrt(p)) вместо O(p).

1. Вычислите `m = ceil(sqrt(p-1))`
2. Baby steps: таблица `{g^j : j}` для `j = 0..m-1`
3. Giant steps: для `i = 0..m-1` проверьте `h * (g^(-m))^i` в таблице
4. Если нашли совпадение: `x = i*m + j`

In [ ]:
# Ваш код здесь


## Челлендж: реализация GF(2^8)

Полиномиальное поле `GF(2^8)` используется в AES (Advanced Encryption Standard). Элементы — полиномы степени до 7 с коэффициентами из {0, 1}.

**Неприводимый полином AES:** `x^8 + x^4 + x^3 + x + 1` (0x11B в hex)

Реализуйте:
1. Сложение: XOR коэффициентов
2. Умножение: полиномиальное умножение по модулю неприводимого полинома
3. Обратный элемент
4. Сравните с `galois.GF(2**8)`

In [ ]:
import galois

class GF256:
    """Элемент поля GF(2^8) = GF(256).
    
    Неприводимый полином: x^8 + x^4 + x^3 + x + 1 (0x11B)
    Элементы представлены как байты (0-255).
    """
    MODULUS = 0x11B  # x^8 + x^4 + x^3 + x + 1
    
    def __init__(self, value: int):
        self.value = value & 0xFF
    
    def __repr__(self):
        return f"GF256(0x{self.value:02x})"
    
    def __add__(self, other):
        # Ваш код здесь
        pass
    
    def __mul__(self, other):
        # Ваш код здесь (Russian Peasant Multiplication)
        pass
    
    def inverse(self):
        # Ваш код здесь
        pass

# Тесты с galois
GF_ref = galois.GF(2**8)
# Сравните результаты вашей реализации с galois